In [78]:
import re
import pandas as pd
import numpy as np

In [79]:
df_reviews = pd.read_csv('https://raw.githubusercontent.com/danmecj/WID3002-NLP-Assignment/main/Preprocessed_Text.csv?token=GHSAT0AAAAAABVJF27HCR77NVKIYXHLWFY6YVIE64A')

df_reviews

,Unnamed: 0,name,locality,Preprocessed
0,0,Dr. Jeyarajah S. @ Psychiatrists (Mental Illne...,Johor Bahru,"['dr', 'jeyarajah', 'experienced', 'kind', 'pr..."
1,1,Dr. Jeyarajah S. @ Psychiatrists (Mental Illne...,Johor Bahru,"['dr', 'nice', 'patient', 'listened', 'problem..."
2,2,Dr. Jeyarajah S. @ Psychiatrists (Mental Illne...,Johor Bahru,"['first', 'reluctant', 'scared', 'see', 'psych..."
3,3,Dr. Jeyarajah S. @ Psychiatrists (Mental Illne...,Johor Bahru,"['experience', 'doctor', 'help', 'giving', 'me..."
4,4,Dr. Jeyarajah S. @ Psychiatrists (Mental Illne...,Johor Bahru,"['dr', 'jeya', 'second', 'psychiatrist', 'find..."
...,...,...,...,...
1592,1592,One Stop Physiotherapy Changlun,Changlun,['excellent']
1593,1593,One Stop Physiotherapy Changlun,Changlun,['good']
1594,1594,One Stop Physiotherapy Changlun,Changlun,"['thank', 'one', 'stop', 'physio', 'tok', 'sto..."
1595,1595,Jabatan Psikiatri @ Pulau Pinang,George Town,"['phone', 'number', 'stated', 'reachable']"


In [80]:
reviews = {}

for i in range(len(df_reviews.index)):
  location = df_reviews.loc[i]
  locality = location['locality']
  name = location['name']
  text = location['Preprocessed']

  if locality not in reviews:
    reviews[locality] = {}

  if name not in reviews[locality]:
    reviews[locality][name] = {'review' : [], 'total_reviews' : 0,
                               'ratings' : {'facilities' : {'positive' : [],
                                                            'negative' : [],
                                                            'score' : [],
                                                            'sigmoid_score' : [],
                                                            'overall_score' : 0},
                                            'services' : {'positive' : [],
                                                          'negative' : [],
                                                          'score' : [],
                                                          'sigmoid_score' : [],
                                                          'overall_score' : 0},
                                            'cost' : {'positive' : [],
                                                      'negative' : [],
                                                      'score' : [],
                                                      'sigmoid_score' : [],
                                                      'overall_score' : 0}}}

In [81]:
for i in range(len(df_reviews.index)):
  location = df_reviews.loc[i]
  locality = location['locality']
  name = location['name']
  text = location['Preprocessed']
  string = ''
  text = []
  for char in location['Preprocessed']:
      check = str(char)
      if re.match(r'[a-zA-Z]',check):
          string += char
            
      if not re.match(r'[a-zA-Z]',check) and len(string) != 0:
          text.append(string)
          string = ''

  reviews[locality][name]['review'].append(text)
  reviews[locality][name]['total_reviews'] += 1

  for category in reviews[locality][name]['ratings']:
    for sub_cat in reviews[locality][name]['ratings'][category]:
      if sub_cat == 'overall_score':
        continue

      
      reviews[locality][name]['ratings'][category][sub_cat].append(0)

In [82]:
facility = {'Asp' : ['design', 'wad', 'clinic', 'ward', 'hospital', 'site', 'website', 'ambiance', 'outside', 'table', 'pharmaceutical', 'complex', 'wheelchair',
                     'unit', 'pharmacy', 'facility', 'dept','system', 'furniture', 'klink', 'loo', 'chair', 'aircon', 'wifi', 'corridor', 'environment', 'department'],
           'Pos' : ['ready', 'operational', 'reputable', 'nice', 'spacious', 'calmer', 'warm', 'systematic', 'gud', 'bagus', 'quality', 'adequate', 'adequately', 'cozy',
                      'accomodating', 'plenty', 'bersih', 'quiet', 'humane', 'clean', 'cleanliness', 'fastest', 'functioning', 'cleaned', 'good', 'hygienic', 'furnished'],
           'Neg' : ['disorganized', 'uncomfortable', 'noise', 'insufficient', 'kurang', 'neglect', 'unpleasant', 'deteriorated', 'bad', 'unsafe', 'teruk', 'neglected',
                      'panas','disgust', 'squat', 'uncleaned']}


cost = {'Asp' : ['value', 'pricing', 'cost', 'financial', 'fee', 'pay', 'price', 'priced', 'paid', 'cash', 'charged', 'charging', 'charge', 'money', 'allocate', 'byr'],
        'Pos' : ['worth', 'affordable', 'gud', 'reasonable', 'moderate', 'moderately', 'ok', 'suitable', 'budget', 'cheaper', 'reasonably', 'enough', 'refunded',
                   'murah', 'afford'],
        'Neg' : ['underpaid', 'squeeze', 'wasted', 'con', 'poor', 'much', 'expensive', 'overcharging', 'overpriced', 'ridiculous', 'costly', 'burden', 'bad', 'teruk',
                   'mahal', 'scammiest']}

service = {'Asp' : ['scheduling', 'counsel', 'dignity', 'etika', 'performance', 'receptionist', 'doct', 'pesakit', 'instruction', 'service', 'certificate', 'talked',
                      'skill', 'technique', 'conversation', 'counsellor', 'protocol', 'availability', 'profession', 'assistant', 'employee', 'consulted', 
                      'pharmaceutical', 'sevices', 'checkup', 'diagnostic', 'treatment', 'discharge', 'ability', 'rehab', 'procedure', 'communication', 'doc', 'help', 
                      'therapy', 'punctuality', 'doctor', 'counselling', 'counseling', 'psychiatrist', 'pychastric', 'reception', 'receptiionist', 'hospitality', 
                      'doktor', 'specialist', 'consulting', 'overall', 'consult', 'psychiatry', 'mentalhealth', 'mental health', 'consultantion', 'shrink'],
           'Pos' : ['properly', 'compassion', 'acceptance', 'pleasantly', 'happier', 'gratitude', 'listen', 'healthy', 'wellness', 'juxtaposed', 'skilled', 'energy', 
                      'graduated', 'encouraged', 'hardworking', 'confident', 'much', 'saved', 'enlightenment', 'secure', 'beloved', 'reputable', 'help', 'fast', 'speedy',
                      'safe', 'empathetic', 'satisfying', 'helpful', 'stable', 'improvement', 'nice', 'appropriate', 'faaasst', 'welcomed', 'pro', 'savvy', 'profesional', 
                      'exceptionally', 'calmer', 'content', 'warm', 'smoothly', 'improving', 'empathise', 'academic', 'systematic', 'gud', 'welcoming', 'convincing', 
                      'expertise', 'assure', 'detailed', 'quick', 'bagus', 'streamline', 'timely', 'observant', 'loving', 'understanding', 'talented', 'satisfactory', 
                      'excelent', 'holistic', 'knowledgeable', 'responsive', 'professionally', 'exceptional', 'sweet', 'reassurance', 'quality', 'diligent', 'ok', 
                      'suitable', 'confidence', 'freindly', 'trusted', 'relaxing', 'helpful', 'humane', 'considerate', 'effective', 'lliberating', 'respectable', 
                      'comprehensive', 'trustworthy', 'kindly', 'helpfull', 'decent', 'perfect', 'confidentiality', 'therapeutic', 'good', 'peramah', 'heaven', 
                      'apreciative', 'satisfied', 'dedication', 'recommended', 'punctual'],
           'Neg' : ['fake', 'stressed', 'scold', 'con', 'ego', 'loudly', 'insane', 'faking', 'beware', 'rude', 'lazy', 'trigerred', 'mad', 'refused', 'hate', 
                      'sarcastic', 'disorganized', 'ridicule', 'overbookings', 'uncomfortable', 'shouting', 'slapped', 'exhausted', 'frustrated', 'bully', 
                      'inconsistent', 'poorly', 'clueless', 'crazy', 'suspicious', 'stupid', 'blabbering', 'rudest', 'unashamedly', 'unpolite', 'disgusted', 
                      'unprofessional', 'totallyridiculous', 'unfruitful', 'dissatisfied', 'snapped', 'worthless', 'neglect', 'ridiculous', 'disinterested', 
                      'threatening', 'humilliation', 'judgemental', 'rudecounterstaff', 'heartless', 'burden', 'dodgy', 'overwhelmed', 'outraged', 
                      'inefficient', 'gaslighting', 'lashed', 'screaming', 'selfish', 'unpreparedness', 'incompetent', 'inompetent', 'impolite', 'bad', 
                      'disrespectful', 'inexperienced', 'rudeness', 'unsafe', 'discriminated', 'teruk', 'fraud', 'offensive', 'fastest', 'doubtful', 'hurtful', 
                      'disappointed', 'neglected', 'inexperience', 'unsatisfying', 'unempathetic', 'ineffective', 'mislead', 'frustrating', 'worrying', 'hell', 
                      'geram', 'fantastic', 'unsatisfactory', 'unhelpful', 'arrogant', 'intimidating', 'disappointment']}

In [83]:
def sigmoid(val):
  a = 1 + np.exp(-val)
  return 5/a

In [84]:
check = 6

for locality in reviews:
  for name in reviews[locality]:
    for index in range(len(reviews[locality][name]['review'])):
      hold = reviews[locality][name]['review'][index]
      num = len(hold)

      for word_idx in range(num):
        back = 0 if word_idx < check else word_idx - check
        front = num if num - check < i else i + check

        pre = hold[back : word_idx + 1]
        focus = hold[word_idx]
        post = hold[word_idx : front]

        if focus in facility['Asp']:
          for word in pre:
            if word in facility['Pos']:
              reviews[locality][name]['ratings']['facilities']['positive'][index] += 1
            if word in facility['Neg']:
              reviews[locality][name]['ratings']['facilities']['negative'][index] += 1

          for word in post:
            if word in facility['Pos']:
              reviews[locality][name]['ratings']['facilities']['positive'][index] += 1
            if word in facility['Neg']:
              reviews[locality][name]['ratings']['facilities']['negative'][index] += 1


        if focus in cost['Asp']:
          for word in pre:
            if word in cost['Pos']:
                reviews[locality][name]['ratings']['cost']['positive'][index] += 1
            if word in cost['Neg']:
                reviews[locality][name]['ratings']['cost']['negative'][index] += 1

          for word in post:
            if word in cost['Pos']:
                reviews[locality][name]['ratings']['cost']['positive'][index] += 1
            if word in cost['Neg']:
                reviews[locality][name]['ratings']['cost']['negative'][index] += 1

        if focus in service['Asp']:
          for word in pre:
            if word in service['Pos']:
                reviews[locality][name]['ratings']['services']['positive'][index] += 1
            if word in service['Neg']:
                reviews[locality][name]['ratings']['services']['negative'][index] += 1

          for word in post:
            if word in service['Pos']:
                reviews[locality][name]['ratings']['services']['positive'][index] += 1
            if word in service['Neg']:
                reviews[locality][name]['ratings']['services']['negative'][index] += 1

        if focus in facility['Neg']:
          reviews[locality][name]['ratings']['facilities']['negative'][index] += 1

        if focus in cost['Neg']:
          reviews[locality][name]['ratings']['cost']['negative'][index] += 1

        if focus in service['Neg']:
          reviews[locality][name]['ratings']['services']['negative'][index] += 1

        if focus in facility['Pos']:
          reviews[locality][name]['ratings']['facilities']['positive'][index] += 1

        if focus in cost['Pos']:
          reviews[locality][name]['ratings']['cost']['positive'][index] += 1

        if focus in service['Pos']:
          reviews[locality][name]['ratings']['services']['positive'][index] += 1

      for c in reviews[locality][name]['ratings']:
        pos = reviews[locality][name]['ratings'][c]['positive'][index]
        neg = reviews[locality][name]['ratings'][c]['negative'][index]
        val = pos/ (neg + 1) if pos >= neg else (neg / (pos + 1)) * -1
        reviews[locality][name]['ratings'][c]['score'][index] = val
        reviews[locality][name]['ratings'][c]['sigmoid_score'][index] = sigmoid(val)

      for c in reviews[locality][name]['ratings']:
        summation = sum(reviews[locality][name]['ratings'][c]['sigmoid_score'])
        length = len(reviews[locality][name]['ratings'][c]['sigmoid_score'])
        reviews[locality][name]['ratings'][c]['overall_score'] = summation / length

In [85]:
overall_score_data = {'name' : [],'locality' : [],
                      'total_reviews' : [],
                      'overall_service_score' : [],
                      'overall_facility_score' : [],
                      'overall_cost_score' : []}

for locality in reviews:
  for name in reviews[locality]:
    overall_score_data['name'].append(name)
    overall_score_data['locality'].append(locality)

    overall_score_data['total_reviews'].append(reviews[locality][name]['total_reviews'])

    hold = reviews[locality][name]['ratings']
    overall_score_data['overall_service_score'].append(hold['services']['overall_score'])
    overall_score_data['overall_facility_score'].append(hold['facilities']['overall_score'])
    overall_score_data['overall_cost_score'].append(hold['cost']['overall_score'])

df_score = pd.DataFrame(data = overall_score_data)

In [86]:
df_score.to_csv('Overall_Category_Score.csv')
df_score.to_excel('Overall_Category_Score.xlsx')

In [87]:
df_score

,name,locality,total_reviews,overall_service_score,overall_facility_score,overall_cost_score
0,Dr. Jeyarajah S. @ Psychiatrists (Mental Illne...,Johor Bahru,10,4.848819,3.444131,2.761973
1,Chan Specialist Clinic,Johor Bahru,10,2.772198,2.500000,2.500000
2,Hospital Permai Johor Bahru,Johor Bahru,15,2.731059,2.880797,2.500000
3,Klinik Pakar Jimmy,Johor Bahru,8,1.458286,2.024004,2.500000
4,Arquila ժՏʍ-ѵ Psychology Consultant,Johor Bahru,10,4.671844,2.962117,2.622459
...,...,...,...,...,...,...
323,Pantai Hospital Laguna Merbok,Sungai Petani,5,3.492972,4.129156,2.268941
324,Klinik Vigneshwer (KIDA),Kulim,5,3.452574,3.230148,2.500000
325,Klinik Kulim,Kulim,5,4.285822,2.982014,2.500000
326,KLINIK PAKAR DR. WEE (Psychiatry/Psychological...,Butterworth,1,2.500000,2.500000,2.500000


In [88]:
df_score.loc[df_score['overall_facility_score'] == 2.5]

,name,locality,total_reviews,overall_service_score,overall_facility_score,overall_cost_score
1,Chan Specialist Clinic,Johor Bahru,10,2.772198,2.5,2.500000
5,Klinik Kaunseling Penawar,Johor Bahru,2,2.500000,2.5,2.500000
11,Klinik Achuta,Johor Bahru,10,2.649738,2.5,3.111856
13,Shinewave Retreat - Rehabilitation centre & ps...,Johor Bahru,2,4.403985,2.5,2.500000
20,Lsy Healthcare,Ulu Tiram,2,2.500000,2.5,2.500000
...,...,...,...,...,...,...
317,Jabatan Psikiatri @ Pulau Pinang,George Town,4,2.500000,2.5,2.500000
319,Clinic Dr. Rosly,Sungai Petani,3,3.639388,2.5,2.885098
321,Lifecare Physiotherapy Center,Sungai Petani,1,2.500000,2.5,2.500000
326,KLINIK PAKAR DR. WEE (Psychiatry/Psychological...,Butterworth,1,2.500000,2.5,2.500000


In [89]:
reviews['Kuala Terengganu']['SALAM Specialist Hospital Kuala Terengganu']['review']

[['third',
  'time',
  'service',
  'great',
  'fact',
  'improved',
  'much',
  'time',
  'time',
  'waiting',
  'time',
  'acceptable',
  'doctor',
  'knowledgeable',
  'solve',
  'medical',
  'issue',
  'big',
  'hospital',
  'well',
  'equipped',
  'doctor',
  'helpful',
  'healthy',
  'thanks'],
 ['reading',
  'bad',
  'review',
  'wonder',
  'also',
  'bad',
  'experience',
  'poor',
  'service',
  'expectation',
  'especially',
  'using',
  'money',
  'like',
  'highly',
  'recommend',
  'cant',
  'believe',
  'fact',
  'government',
  'hospital',
  'far',
  'better',
  'salam',
  'hospital',
  'staff',
  'guard',
  'unprofessional',
  'ethical',
  'patient',
  'guardian',
  'think',
  'management',
  'train',
  'manner',
  'dealing',
  'customer'],
 ['bad',
  'service',
  'highly',
  'recommended',
  'cant',
  'believe',
  'fact',
  'government',
  'hospital',
  'far',
  'better',
  'salam',
  'specialist',
  'term',
  'staff',
  'service',
  'especially',
  'staff',
  'covid',